In [1]:
import findspark

findspark.init("/opt/bitnami/spark")
findspark.find()

from pyspark.sql import SparkSession

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [3]:
import dotenv
from os import getenv

dotenv.find_dotenv()
dotenv.load_dotenv()

False

In [4]:
spark = (
    SparkSession.builder.master("spark://spark-master:7077")
    .appName("APP")
    .config(
        map={
            "spark.hadoop.fs.s3a.access.key": getenv("S3_ACCESS_KEY_ID"),
            "spark.hadoop.fs.s3a.secret.key": getenv("S3_SECRET_ACCESS_KEY"),
            "spark.hadoop.fs.s3a.endpoint": getenv("S3_ENDPOINT_URL"),
            "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
        }
    )
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
[2023-09-14 16:42:13] {NativeCodeLoader} WARN: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [55]:
df = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message"
).where(F.col("date") == "2022-01-15")


In [10]:
df.show(10)

+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|  admins|channel_id|           datetime|media_type|           media_src|             message|message_channel_to|message_from|message_group|message_id|message_to|message_ts|reaction_from|reaction_type|subscription_channel|subscription_user|                tags|user|                lat|               lon|      date|
+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|[141644]|    235517|2022-01-15 06:10:05|     ima

In [14]:
df.explain(mode='cost')

== Optimized Logical Plan ==
Filter (isnotnull(date#20) AND (date#20 = 2022-01-15)), Statistics(sizeInBytes=149.0 MiB)
+- Relation [admins#0,channel_id#1L,datetime#2,media_type#3,media_src#4,message#5,message_channel_to#6L,message_from#7L,message_group#8L,message_id#9L,message_to#10L,message_ts#11,reaction_from#12,reaction_type#13,subscription_channel#14L,subscription_user#15,tags#16,user#17,lat#18,lon#19,date#20] parquet, Statistics(sizeInBytes=149.0 MiB)

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [admins#0,channel_id#1L,datetime#2,media_type#3,media_src#4,message#5,message_channel_to#6L,message_from#7L,message_group#8L,message_id#9L,message_to#10L,message_ts#11,reaction_from#12,reaction_type#13,subscription_channel#14L,subscription_user#15,tags#16,user#17,lat#18,lon#19,date#20] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_ty..., PartitionFilters: [isnotnull(da

In [12]:
df.explain(mode="formatted")

== Physical Plan ==
* ColumnarToRow (2)
+- Scan parquet  (1)


(1) Scan parquet 
Output [21]: [admins#0, channel_id#1L, datetime#2, media_type#3, media_src#4, message#5, message_channel_to#6L, message_from#7L, message_group#8L, message_id#9L, message_to#10L, message_ts#11, reaction_from#12, reaction_type#13, subscription_channel#14L, subscription_user#15, tags#16, user#17, lat#18, lon#19, date#20]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#20), (date#20 = 2022-01-15)]
ReadSchema: struct<admins:array<bigint>,channel_id:bigint,datetime:timestamp,media_type:string,media_src:string,message:string,message_channel_to:bigint,message_from:bigint,message_group:bigint,message_id:bigint,message_to:bigint,message_ts:timestamp,reaction_from:string,reaction_type:string,subscription_channel:bigint,subscription_user:string,tags:array<string>,user:string,lat:double,lon:double>

(2) Column

In [13]:
df.explain(mode="extended")

== Parsed Logical Plan ==
'Filter ('date = 2022-01-15)
+- Relation [admins#0,channel_id#1L,datetime#2,media_type#3,media_src#4,message#5,message_channel_to#6L,message_from#7L,message_group#8L,message_id#9L,message_to#10L,message_ts#11,reaction_from#12,reaction_type#13,subscription_channel#14L,subscription_user#15,tags#16,user#17,lat#18,lon#19,date#20] parquet

== Analyzed Logical Plan ==
admins: array<bigint>, channel_id: bigint, datetime: timestamp, media_type: string, media_src: string, message: string, message_channel_to: bigint, message_from: bigint, message_group: bigint, message_id: bigint, message_to: bigint, message_ts: timestamp, reaction_from: string, reaction_type: string, subscription_channel: bigint, subscription_user: string, tags: array<string>, user: string, lat: double, lon: double, date: date
Filter (date#20 = cast(2022-01-15 as date))
+- Relation [admins#0,channel_id#1L,datetime#2,media_type#3,media_src#4,message#5,message_channel_to#6L,message_from#7L,message_group#

In [16]:
df.printSchema()

root
 |-- admins: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- channel_id: long (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- media_type: string (nullable = true)
 |-- media_src: string (nullable = true)
 |-- message: string (nullable = true)
 |-- message_channel_to: long (nullable = true)
 |-- message_from: long (nullable = true)
 |-- message_group: long (nullable = true)
 |-- message_id: long (nullable = true)
 |-- message_to: long (nullable = true)
 |-- message_ts: timestamp (nullable = true)
 |-- reaction_from: string (nullable = true)
 |-- reaction_type: string (nullable = true)
 |-- subscription_channel: long (nullable = true)
 |-- subscription_user: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- date: date (nullable = true)



In [17]:
df.show(10)

+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|  admins|channel_id|           datetime|media_type|           media_src|             message|message_channel_to|message_from|message_group|message_id|message_to|message_ts|reaction_from|reaction_type|subscription_channel|subscription_user|                tags|user|                lat|               lon|      date|
+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|[141644]|    235517|2022-01-15 06:10:05|     ima

In [49]:
df = (
    df
        .withColumn('admins', F.concat_ws(",", F.col("admins")))
        .withColumn('tags', F.concat_ws(",", F.col("tags")))
)

In [25]:
df.printSchema()

root
 |-- admins: string (nullable = false)
 |-- channel_id: long (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- media_type: string (nullable = true)
 |-- media_src: string (nullable = true)
 |-- message: string (nullable = true)
 |-- message_channel_to: long (nullable = true)
 |-- message_from: long (nullable = true)
 |-- message_group: long (nullable = true)
 |-- message_id: long (nullable = true)
 |-- message_to: long (nullable = true)
 |-- message_ts: timestamp (nullable = true)
 |-- reaction_from: string (nullable = true)
 |-- reaction_type: string (nullable = true)
 |-- subscription_channel: long (nullable = true)
 |-- subscription_user: string (nullable = true)
 |-- tags: string (nullable = false)
 |-- user: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- date: date (nullable = true)



In [42]:
spark.conf.set("spark.hadoop.mapreduce.outputcommitter.factory.scheme.s3a",
               "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory")
spark.conf.set("spark.hadoop.mapreduce.outputcommitter.factory.scheme.s3",
               "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory")


spark.conf.set("spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled", True)


In [56]:
# df.write.partitionBy('date').csv(
#     "s3a://data-ice-lake-05/master/tmp/events/event_type=message", mode="overwrite", header=True, compression='gzip', sep="|"
# )

df.write.partitionBy('date').parquet(
    "s3a://data-ice-lake-05/master/tmp/events/event_type=message", mode="overwrite", compression='gzip'
)

In [29]:
df.explain(mode="formatted")

== Physical Plan ==
* Project (3)
+- * ColumnarToRow (2)
   +- Scan parquet  (1)


(1) Scan parquet 
Output [21]: [admins#0, channel_id#1L, datetime#2, media_type#3, media_src#4, message#5, message_channel_to#6L, message_from#7L, message_group#8L, message_id#9L, message_to#10L, message_ts#11, reaction_from#12, reaction_type#13, subscription_channel#14L, subscription_user#15, tags#16, user#17, lat#18, lon#19, date#20]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#20), (date#20 = 2022-01-15)]
ReadSchema: struct<admins:array<bigint>,channel_id:bigint,datetime:timestamp,media_type:string,media_src:string,message:string,message_channel_to:bigint,message_from:bigint,message_group:bigint,message_id:bigint,message_to:bigint,message_ts:timestamp,reaction_from:string,reaction_type:string,subscription_channel:bigint,subscription_user:string,tags:array<string>,user:string,lat:double,lon

In [57]:
# df = spark.read.csv("s3a://data-ice-lake-05/master/tmp/events/event_type=message", header=True,  sep="|").where(F.col('date') == "2022-01-15")

df = spark.read.parquet("s3a://data-ice-lake-05/master/tmp/events/event_type=message").where(F.col('date') == "2022-01-15")

In [52]:
df.show(10)

+--------------------+----------+--------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+----+----+----+----+----------+
|              admins|channel_id|            datetime|media_type|           media_src|             message|message_channel_to|message_from|message_group|message_id|message_to|message_ts|reaction_from|reaction_type|subscription_channel|subscription_user|tags|user| lat| lon|      date|
+--------------------+----------+--------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+----+----+----+----+----------+
|              141644|    235517|2022-01-15T06:10:...|     image|https://company.c...|In the Silicon Va...|              null|        null|      

In [53]:
# csv
df.explain('formatted')

== Physical Plan ==
Scan csv  (1)


(1) Scan csv 
Output [21]: [admins#1401, channel_id#1402, datetime#1403, media_type#1404, media_src#1405, message#1406, message_channel_to#1407, message_from#1408, message_group#1409, message_id#1410, message_to#1411, message_ts#1412, reaction_from#1413, reaction_type#1414, subscription_channel#1415, subscription_user#1416, tags#1417, user#1418, lat#1419, lon#1420, date#1421]
Batched: false
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/tmp/events/event_type=message]
PartitionFilters: [isnotnull(date#1421), (date#1421 = 2022-01-15)]
ReadSchema: struct<admins:string,channel_id:string,datetime:string,media_type:string,media_src:string,message:string,message_channel_to:string,message_from:string,message_group:string,message_id:string,message_to:string,message_ts:string,reaction_from:string,reaction_type:string,subscription_channel:string,subscription_user:string,tags:string,user:string,lat:string,lon:string>




In [58]:
# parquet

df.explain('formatted')

== Physical Plan ==
* ColumnarToRow (2)
+- Scan parquet  (1)


(1) Scan parquet 
Output [21]: [admins#1657, channel_id#1658L, datetime#1659, media_type#1660, media_src#1661, message#1662, message_channel_to#1663L, message_from#1664L, message_group#1665L, message_id#1666L, message_to#1667L, message_ts#1668, reaction_from#1669, reaction_type#1670, subscription_channel#1671L, subscription_user#1672, tags#1673, user#1674, lat#1675, lon#1676, date#1677]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/tmp/events/event_type=message]
PartitionFilters: [isnotnull(date#1677), (date#1677 = 2022-01-15)]
ReadSchema: struct<admins:array<bigint>,channel_id:bigint,datetime:timestamp,media_type:string,media_src:string,message:string,message_channel_to:bigint,message_from:bigint,message_group:bigint,message_id:bigint,message_to:bigint,message_ts:timestamp,reaction_from:string,reaction_type:string,subscription_channel:bigint,subscription_user:string,tags:array<string>,user:string

In [5]:
df = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message", mergeSchema=True
).where((F.col("date") >= "2022-01-15") & (F.col('date') <= "2022-05-15"))

[2023-09-14 16:56:49] {MetricsConfig} WARN: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [66]:
df.show(10)

+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|  admins|channel_id|           datetime|media_type|           media_src|             message|message_channel_to|message_from|message_group|message_id|message_to|message_ts|reaction_from|reaction_type|subscription_channel|subscription_user|                tags|user|                lat|               lon|      date|
+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|[137449]|     97347|2022-04-15 08:54:15|     ima

In [61]:
df.explain(True)

== Parsed Logical Plan ==
'Filter (('date >= 2022-01-15) AND ('date <= 2022-02-15))
+- Relation [admins#1700,channel_id#1701L,datetime#1702,media_type#1703,media_src#1704,message#1705,message_channel_to#1706L,message_from#1707L,message_group#1708L,message_id#1709L,message_to#1710L,message_ts#1711,reaction_from#1712,reaction_type#1713,subscription_channel#1714L,subscription_user#1715,tags#1716,user#1717,lat#1718,lon#1719,date#1720] parquet

== Analyzed Logical Plan ==
admins: array<bigint>, channel_id: bigint, datetime: timestamp, media_type: string, media_src: string, message: string, message_channel_to: bigint, message_from: bigint, message_group: bigint, message_id: bigint, message_to: bigint, message_ts: timestamp, reaction_from: string, reaction_type: string, subscription_channel: bigint, subscription_user: string, tags: array<string>, user: string, lat: double, lon: double, date: date
Filter ((date#1720 >= cast(2022-01-15 as date)) AND (date#1720 <= cast(2022-02-15 as date)))
+- R

In [8]:
df.rdd.getNumPartitions()

6

In [6]:
df.explain(mode='formatted')

== Physical Plan ==
* ColumnarToRow (2)
+- Scan parquet  (1)


(1) Scan parquet 
Output [21]: [admins#0, channel_id#1L, datetime#2, media_type#3, media_src#4, message#5, message_channel_to#6L, message_from#7L, message_group#8L, message_id#9L, message_to#10L, message_ts#11, reaction_from#12, reaction_type#13, subscription_channel#14L, subscription_user#15, tags#16, user#17, lat#18, lon#19, date#20]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#20), (date#20 >= 2022-01-15), (date#20 <= 2022-05-15)]
ReadSchema: struct<admins:array<bigint>,channel_id:bigint,datetime:timestamp,media_type:string,media_src:string,message:string,message_channel_to:bigint,message_from:bigint,message_group:bigint,message_id:bigint,message_to:bigint,message_ts:timestamp,reaction_from:string,reaction_type:string,subscription_channel:bigint,subscription_user:string,tags:array<string>,user:string,lat:doub

In [10]:
df.storageLevel

StorageLevel(False, False, False, False, 1)

In [11]:
df = df.select('channel_id').distinct()

== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Project (2)
            +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [channel_id#1L, date#20]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#20), (date#20 >= 2022-01-15), (date#20 <= 2022-05-15)]
ReadSchema: struct<channel_id:bigint>

(2) Project
Output [1]: [channel_id#1L]
Input [2]: [channel_id#1L, date#20]

(3) HashAggregate
Input [1]: [channel_id#1L]
Keys [1]: [channel_id#1L]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#1L]

(4) Exchange
Input [1]: [channel_id#1L]
Arguments: hashpartitioning(channel_id#1L, 200), ENSURE_REQUIREMENTS, [plan_id=20]

(5) HashAggregate
Input [1]: [channel_id#1L]
Keys [1]: [channel_id#1L]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#1L]

(6) AdaptiveSparkPlan
Output [1]: [channel_

In [12]:
df = df.select('channel_id').distinct()

In [14]:
df.show(10, False)

+----------+
|channel_id|
+----------+
|618771    |
|643321    |
|776940    |
|319568    |
|671323    |
|149806    |
|405912    |
|347525    |
|122127    |
|370471    |
+----------+
only showing top 10 rows



In [16]:
df.explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Project (2)
            +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [channel_id#1L, date#20]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#20), (date#20 >= 2022-01-15), (date#20 <= 2022-05-15)]
ReadSchema: struct<channel_id:bigint>

(2) Project
Output [1]: [channel_id#1L]
Input [2]: [channel_id#1L, date#20]

(3) HashAggregate
Input [1]: [channel_id#1L]
Keys [1]: [channel_id#1L]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#1L]

(4) Exchange
Input [1]: [channel_id#1L]
Arguments: hashpartitioning(channel_id#1L, 200), ENSURE_REQUIREMENTS, [plan_id=35]

(5) HashAggregate
Input [1]: [channel_id#1L]
Keys [1]: [channel_id#1L]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#1L]

(6) AdaptiveSparkPlan
Output [1]: [channel_

In [17]:
df.rdd.getNumPartitions()

1

In [38]:
df = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message", mergeSchema=True
).where((F.col("date") >= "2022-01-15") & (F.col('date') <= "2022-05-15"))

In [22]:
df = df.select('channel_id').distinct()

In [23]:
df.explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Project (2)
            +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [channel_id#101L, date#120]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#120), (date#120 >= 2022-01-15), (date#120 <= 2022-05-15)]
ReadSchema: struct<channel_id:bigint>

(2) Project
Output [1]: [channel_id#101L]
Input [2]: [channel_id#101L, date#120]

(3) HashAggregate
Input [1]: [channel_id#101L]
Keys [1]: [channel_id#101L]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#101L]

(4) Exchange
Input [1]: [channel_id#101L]
Arguments: hashpartitioning(channel_id#101L, 200), ENSURE_REQUIREMENTS, [plan_id=247]

(5) HashAggregate
Input [1]: [channel_id#101L]
Keys [1]: [channel_id#101L]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#101L]

(6) AdaptiveSpa

In [39]:
df.show(10)

+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|  admins|channel_id|           datetime|media_type|           media_src|             message|message_channel_to|message_from|message_group|message_id|message_to|message_ts|reaction_from|reaction_type|subscription_channel|subscription_user|                tags|user|                lat|               lon|      date|
+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|[137449]|     97347|2022-04-15 08:54:15|     ima

In [40]:
df = df.drop('message')

In [41]:
df = (
    df
        .withColumn('admins', F.concat_ws(",", F.col("admins")))
        .withColumn('tags', F.concat_ws(",", F.col("tags")))
)

In [42]:
df.write.partitionBy('date').csv("s3a://data-ice-lake-05/master/tmp/events/event_type=message", header=True, sep=';', mode='overwrite')

[2023-09-14 17:24:20] {AbstractS3ACommitterFactory} WARN: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [47]:
df = spark.read.csv("s3a://data-ice-lake-05/master/tmp/events/event_type=message",  header=True, sep=';')

In [44]:
df.show(10)

+------+----------+--------------------+----------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|admins|channel_id|            datetime|media_type|           media_src|message_channel_to|message_from|message_group|message_id|message_to|message_ts|reaction_from|reaction_type|subscription_channel|subscription_user|                tags|user|                lat|               lon|      date|
+------+----------+--------------------+----------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|   604|    703237|2022-02-14T12:57:...|     image|https://company.c...|            703237|         604|         nu

In [45]:
df = df.select('channel_id').distinct().explain(mode='formatted')

In [49]:

df.select('channel_id').distinct().explain(True)

== Parsed Logical Plan ==
Deduplicate [channel_id#1200]
+- Project [channel_id#1200]
   +- Relation [admins#1199,channel_id#1200,datetime#1201,media_type#1202,media_src#1203,message_channel_to#1204,message_from#1205,message_group#1206,message_id#1207,message_to#1208,message_ts#1209,reaction_from#1210,reaction_type#1211,subscription_channel#1212,subscription_user#1213,tags#1214,user#1215,lat#1216,lon#1217,date#1218] csv

== Analyzed Logical Plan ==
channel_id: string
Deduplicate [channel_id#1200]
+- Project [channel_id#1200]
   +- Relation [admins#1199,channel_id#1200,datetime#1201,media_type#1202,media_src#1203,message_channel_to#1204,message_from#1205,message_group#1206,message_id#1207,message_to#1208,message_ts#1209,reaction_from#1210,reaction_type#1211,subscription_channel#1212,subscription_user#1213,tags#1214,user#1215,lat#1216,lon#1217,date#1218] csv

== Optimized Logical Plan ==
Aggregate [channel_id#1200], [channel_id#1200]
+- Project [channel_id#1200]
   +- Relation [admins#119

In [46]:
df.explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (6)
+- HashAggregate (5)
   +- Exchange (4)
      +- HashAggregate (3)
         +- Project (2)
            +- Scan csv  (1)


(1) Scan csv 
Output [2]: [channel_id#1040, date#1058]
Batched: false
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/tmp/events/event_type=message]
ReadSchema: struct<channel_id:string>

(2) Project
Output [1]: [channel_id#1040]
Input [2]: [channel_id#1040, date#1058]

(3) HashAggregate
Input [1]: [channel_id#1040]
Keys [1]: [channel_id#1040]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#1040]

(4) Exchange
Input [1]: [channel_id#1040]
Arguments: hashpartitioning(channel_id#1040, 200), ENSURE_REQUIREMENTS, [plan_id=547]

(5) HashAggregate
Input [1]: [channel_id#1040]
Keys [1]: [channel_id#1040]
Functions: []
Aggregate Attributes: []
Results [1]: [channel_id#1040]

(6) AdaptiveSparkPlan
Output [1]: [channel_id#1040]
Arguments: isFinalPlan=false




In [50]:
df1 = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message"
).where(F.col("date") == "2022-01-15")


df2 = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message"
).where(F.col("date") == "2022-01-16")

In [54]:
df1.union(df2).where(F.size(F.col('tags')) > 1).select('channel_id').distinct().explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (11)
+- HashAggregate (10)
   +- Exchange (9)
      +- HashAggregate (8)
         +- Union (7)
            :- Project (3)
            :  +- Filter (2)
            :     +- Scan parquet  (1)
            +- Project (6)
               +- Filter (5)
                  +- Scan parquet  (4)


(1) Scan parquet 
Output [3]: [channel_id#1243L, tags#1258, date#1262]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#1262), (date#1262 = 2022-01-15)]
ReadSchema: struct<channel_id:bigint,tags:array<string>>

(2) Filter
Input [3]: [channel_id#1243L, tags#1258, date#1262]
Condition : (size(tags#1258, true) > 1)

(3) Project
Output [1]: [channel_id#1243L]
Input [3]: [channel_id#1243L, tags#1258, date#1262]

(4) Scan parquet 
Output [3]: [channel_id#1286L, tags#1301, date#1305]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/sourc

In [55]:
file = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message"
).where(F.col("date") == "2022-01-15")

In [57]:
file.show(10)

+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|  admins|channel_id|           datetime|media_type|           media_src|             message|message_channel_to|message_from|message_group|message_id|message_to|message_ts|reaction_from|reaction_type|subscription_channel|subscription_user|                tags|user|                lat|               lon|      date|
+--------+----------+-------------------+----------+--------------------+--------------------+------------------+------------+-------------+----------+----------+----------+-------------+-------------+--------------------+-----------------+--------------------+----+-------------------+------------------+----------+
|[141644]|    235517|2022-01-15 06:10:05|     ima

In [59]:
df1 = (
    file.select('channel_id', 'tags', 'message_id')
)

df2 = (file.select('channel_id', 'admins', 'datetime'))

In [60]:
df1.join(df2, how='inner', on='channel_id').explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Project (12)
   +- SortMergeJoin Inner (11)
      :- Sort (5)
      :  +- Exchange (4)
      :     +- Project (3)
      :        +- Filter (2)
      :           +- Scan parquet  (1)
      +- Sort (10)
         +- Exchange (9)
            +- Project (8)
               +- Filter (7)
                  +- Scan parquet  (6)


(1) Scan parquet 
Output [4]: [channel_id#1414L, message_id#1422L, tags#1429, date#1433]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#1433), (date#1433 = 2022-01-15)]
PushedFilters: [IsNotNull(channel_id)]
ReadSchema: struct<channel_id:bigint,message_id:bigint,tags:array<string>>

(2) Filter
Input [4]: [channel_id#1414L, message_id#1422L, tags#1429, date#1433]
Condition : isnotnull(channel_id#1414L)

(3) Project
Output [3]: [channel_id#1414L, tags#1429, message_id#1422L]
Input [4]: [channel_id#1414L, message_id#

In [61]:
from pyspark.storagelevel import StorageLevel

In [62]:
file = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message"
).where(F.col("date") == "2022-01-15").where(F.col('channel_id').isNotNull())

file.persist(StorageLevel.MEMORY_ONLY)

df1 = (
    file.select('channel_id', 'tags', 'message_id')
)

df2 = (file.select('channel_id', 'admins', 'datetime'))

In [63]:
df1.join(df2, how='inner', on='channel_id').explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (19)
+- Project (18)
   +- SortMergeJoin Inner (17)
      :- Sort (8)
      :  +- Exchange (7)
      :     +- Filter (6)
      :        +- InMemoryTableScan (1)
      :              +- InMemoryRelation (2)
      :                    +- * Filter (5)
      :                       +- * ColumnarToRow (4)
      :                          +- Scan parquet  (3)
      +- Sort (16)
         +- Exchange (15)
            +- Filter (14)
               +- InMemoryTableScan (9)
                     +- InMemoryRelation (10)
                           +- * Filter (13)
                              +- * ColumnarToRow (12)
                                 +- Scan parquet  (11)


(1) InMemoryTableScan
Output [3]: [channel_id#1663L, tags#1678, message_id#1671L]
Arguments: [channel_id#1663L, tags#1678, message_id#1671L], [isnotnull(channel_id#1663L)]

(2) InMemoryRelation
Arguments: [admins#1662, channel_id#1663L, datetime#1664, media_type#1665, media_src#1666, message#

In [64]:
file = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message"
).where(F.col("date") == "2022-01-15").where(F.col('channel_id').isNotNull())

file.cache()

df1 = (
    file.select('channel_id', 'tags', 'message_id')
)

df2 = (file.select('channel_id', 'admins', 'datetime'))

[2023-09-14 18:27:59] {CacheManager} WARN: Asked to cache already cached data.  


In [65]:
df1.join(df2, how='inner', on='channel_id').explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (19)
+- Project (18)
   +- SortMergeJoin Inner (17)
      :- Sort (8)
      :  +- Exchange (7)
      :     +- Filter (6)
      :        +- InMemoryTableScan (1)
      :              +- InMemoryRelation (2)
      :                    +- * Filter (5)
      :                       +- * ColumnarToRow (4)
      :                          +- Scan parquet  (3)
      +- Sort (16)
         +- Exchange (15)
            +- Filter (14)
               +- InMemoryTableScan (9)
                     +- InMemoryRelation (10)
                           +- * Filter (13)
                              +- * ColumnarToRow (12)
                                 +- Scan parquet  (11)


(1) InMemoryTableScan
Output [3]: [channel_id#2474L, tags#2489, message_id#2482L]
Arguments: [channel_id#2474L, tags#2489, message_id#2482L], [isnotnull(channel_id#2474L)]

(2) InMemoryRelation
Arguments: [admins#2473, channel_id#2474L, datetime#2475, media_type#2476, media_src#2477, message#

In [66]:
file = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message"
).where(F.col("date") == "2022-01-15").where(F.col('channel_id').isNotNull())

file.explain(mode='formatted')

== Physical Plan ==
InMemoryTableScan (1)
   +- InMemoryRelation (2)
         +- * Filter (5)
            +- * ColumnarToRow (4)
               +- Scan parquet  (3)


(1) InMemoryTableScan
Output [21]: [admins#3179, channel_id#3180L, datetime#3181, media_type#3182, media_src#3183, message#3184, message_channel_to#3185L, message_from#3186L, message_group#3187L, message_id#3188L, message_to#3189L, message_ts#3190, reaction_from#3191, reaction_type#3192, subscription_channel#3193L, subscription_user#3194, tags#3195, user#3196, lat#3197, lon#3198, date#3199]
Arguments: [admins#3179, channel_id#3180L, datetime#3181, media_type#3182, media_src#3183, message#3184, message_channel_to#3185L, message_from#3186L, message_group#3187L, message_id#3188L, message_to#3189L, message_ts#3190, reaction_from#3191, reaction_type#3192, subscription_channel#3193L, subscription_user#3194, tags#3195, user#3196, lat#3197, lon#3198, date#3199]

(2) InMemoryRelation
Arguments: [admins#3179, channel_id#3180L, date

In [67]:
df1.join(df2, how='cross').explain(mode='formatted')

== Physical Plan ==
CartesianProduct Inner (11)
:- InMemoryTableScan (1)
:     +- InMemoryRelation (2)
:           +- * Filter (5)
:              +- * ColumnarToRow (4)
:                 +- Scan parquet  (3)
+- InMemoryTableScan (6)
      +- InMemoryRelation (7)
            +- * Filter (10)
               +- * ColumnarToRow (9)
                  +- Scan parquet  (8)


(1) InMemoryTableScan
Output [3]: [channel_id#2474L, tags#2489, message_id#2482L]
Arguments: [channel_id#2474L, tags#2489, message_id#2482L]

(2) InMemoryRelation
Arguments: [admins#2473, channel_id#2474L, datetime#2475, media_type#2476, media_src#2477, message#2478, message_channel_to#2479L, message_from#2480L, message_group#2481L, message_id#2482L, message_to#2483L, message_ts#2484, reaction_from#2485, reaction_type#2486, subscription_channel#2487L, subscription_user#2488, tags#2489, user#2490, lat#2491, lon#2492, date#2493], CachedRDDBuilder(org.apache.spark.sql.execution.columnar.DefaultCachedBatchSerializer@5529031,S

In [68]:
file.unpersist()

DataFrame[admins: array<bigint>, channel_id: bigint, datetime: timestamp, media_type: string, media_src: string, message: string, message_channel_to: bigint, message_from: bigint, message_group: bigint, message_id: bigint, message_to: bigint, message_ts: timestamp, reaction_from: string, reaction_type: string, subscription_channel: bigint, subscription_user: string, tags: array<string>, user: string, lat: double, lon: double, date: date]

In [69]:
df = spark.read.parquet(
    "s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message", mergeSchema=True
).where((F.col("date") >= "2022-01-15") & (F.col('date') <= "2022-05-15") & (F.col('message_from').isNotNull())).select("message_from", 'message_id', 'message_ts', 'datetime', 'lat','lon')

In [71]:
df = (
    df
        .withColumn(
                    "msg_ts",
                    F.when(F.col("message_ts").isNotNull(), F.col("message_ts")).otherwise(
                        F.col("datetime")
                    ),
                )
                .drop("message_ts", "datetime")
        )

In [72]:
df.explain(mode='formatted')

== Physical Plan ==
* Project (4)
+- * Filter (3)
   +- * ColumnarToRow (2)
      +- Scan parquet  (1)


(1) Scan parquet 
Output [7]: [datetime#4197, message_from#4202L, message_id#4204L, message_ts#4206, lat#4213, lon#4214, date#4215]
Batched: true
Location: InMemoryFileIndex [s3a://data-ice-lake-05/master/data/source/messenger-yp/events/event_type=message]
PartitionFilters: [isnotnull(date#4215), (date#4215 >= 2022-01-15), (date#4215 <= 2022-05-15)]
PushedFilters: [IsNotNull(message_from)]
ReadSchema: struct<datetime:timestamp,message_from:bigint,message_id:bigint,message_ts:timestamp,lat:double,lon:double>

(2) ColumnarToRow [codegen id : 1]
Input [7]: [datetime#4197, message_from#4202L, message_id#4204L, message_ts#4206, lat#4213, lon#4214, date#4215]

(3) Filter [codegen id : 1]
Input [7]: [datetime#4197, message_from#4202L, message_id#4204L, message_ts#4206, lat#4213, lon#4214, date#4215]
Condition : isnotnull(message_from#4202L)

(4) Project [codegen id : 1]
Output [5]: [messag